In [1]:
import datetime
import math
import pprint
import time

import ee
from IPython.display import Image

import openet
import openet.disalexi.disalexi as disalexi
import openet.disalexi.landsat as landsat
import openet.disalexi.utils as utils

ee.Initialize()

In [2]:
# start_date = '2015-07-11'
# end_date = '2015-07-12'
start_date = '2015-07-01'
end_date = '2015-08-01'

ta_landsat_id = 'projects/disalexi/ta/landsat'
ta_coarse_id = 'projects/disalexi/ta/CONUS'

asset_dt_fmt = '%Y%m%d'
asset_id_fmt = '{collection_id}/{date}'
task_id_fmt = 'disalexi_tair_conus_{date}'

start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

### ALEXI Properties

In [3]:
alexi_coll = ee.ImageCollection('projects/disalexi/alexi/CONUS')
# print(ee.Image(alexi_coll.first()).projection().getInfo()['transform'])
# print(ee.Image(alexi_coll.first()).projection().getInfo()['crs'])
# print(ee.Image(alexi_coll.first()).getInfo()['bands'][0]['dimensions'])
alexi_cs = 0.04
alexi_geo = [0.04, 0.0, -125.04, 0.0, -0.04, 49.82]

alexi_shape = [1456, 625]
alexi_crs = 'EPSG:4326'

alexi_extent = [
    alexi_geo[2], alexi_geo[5] + alexi_geo[4] * alexi_shape[1], 
    alexi_geo[2] + alexi_geo[0] * alexi_shape[0], alexi_geo[5]]
alexi_region = ee.Geometry.Rectangle(alexi_extent, alexi_crs, False) \
    .bounds(1, 'EPSG:4326').coordinates().getInfo()
# print(alexi_extent)
# print(alexi_region)

alexi_geo_str = '[' + ','.join(list(map(str, alexi_geo))) + ']'
alexi_shape_str = '{0}x{1}'.format(*alexi_shape)
# print(alexi_geo_str)
# print(alexi_shape_str)

### Study Area Properties

In [4]:
# Input parameters
output_crs = 'EPSG:4326'

# output_cs = 0.001   # ~100m
output_cs = 0.002   # ~100m
# output_cs = 0.005   # Internal Error
# output_cs = 0.01   # Internal Error
# output_cs = 0.04   # Internal Error
# output_cs = alexi_geo[0]

# output_extent = [-121.9, 38.8, -121.7, 38.9]  # Study Area
# output_extent = [-122.0, 38.7, -121.6, 39.0]  # Study Area
# output_extent = [-122.0, 38.0, -121.0, 39.0]  # 1 x 1 deg
output_extent = [-123.0, 37.8, -120.4, 40.0]  # LC08_044033_20150711
# output_extent = [-123, 35, -118.5, 40]  # Central Valley
# output_extent = [-125, 32, -114, 42]  # California / Nevada
# output_extent = [-125, 25, -65, 50]  # CONUS

# Computed parameters
output_geom = ee.Geometry.Rectangle(output_extent, output_crs, False)
output_region = output_geom.bounds(1, output_crs).coordinates().getInfo()[0][:-1]
output_xmin = min(x for x, y in output_region)
output_ymin = min(y for x, y in output_region)
output_xmax = max(x for x, y in output_region)
output_ymax = max(y for x, y in output_region)
# Expand extent when snapping
output_xmin = math.floor((output_xmin - alexi_extent[0]) / output_cs) * output_cs + alexi_extent[0]
output_ymin = math.floor((output_ymin - alexi_extent[3]) / output_cs) * output_cs + alexi_extent[3]
output_xmax = math.ceil((output_xmax - alexi_extent[0]) / output_cs) * output_cs + alexi_extent[0]
output_ymax = math.ceil((output_ymax - alexi_extent[3]) / output_cs) * output_cs + alexi_extent[3]
# Limit output extent to ALEXI extent for now
output_xmin = max(output_xmin, alexi_extent[0])
output_ymin = max(output_ymin, alexi_extent[1])
output_xmax = min(output_xmax, alexi_extent[2])
output_ymax = min(output_ymax, alexi_extent[3])
output_extent = [output_xmin, output_ymin, output_xmax, output_ymax]
output_geo = [output_cs, 0.0, output_xmin, 0.0, -output_cs, output_ymax]
# output_geo = [alexi_geo[0], 0.0, output_xmin, 0.0, alexi_geo[4], output_ymax]
output_geo_str = '[' + ','.join(list(map(str, output_geo))) + ']'
output_shape_str = '{0}x{1}'.format(
    int(abs(output_extent[2] - output_extent[0]) / output_cs),
    int(abs(output_extent[3] - output_extent[1]) / output_cs))
output_cs_str = '{:0.3f}'.format(output_cs).replace('.', 'p')
print(output_geo_str)
print(output_shape_str)
print(output_cs_str)

[0.002,0.0,-123.0,0.0,-0.002,40.0]
1300x1100
0p002


### Landsat Image, Collection, and Properties

In [5]:
landsat_id = 'LC08_044033_20150711'
landsat_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_044033_20150711')

landsat_crs = landsat_img.select(['B2']).projection().getInfo()['crs']
landsat_geo = landsat_img.select(['B2']).projection().getInfo()['transform']
landsat_shape = landsat_img.select(['B2']).getInfo()['bands'][0]['dimensions']
landsat_geo_str = '[' + ','.join(list(map(str, landsat_geo))) + ']'
landsat_shape_str = '{0}x{1}'.format(*landsat_shape)
print(landsat_crs)
print(landsat_geo_str)
print(landsat_shape_str)

EPSG:32610
[30.0,0.0,499785.0,0.0,-30.0,4423215.0]
7671x7791


In [6]:
landsat_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA') \
    .filterDate(start_date, end_date) \
    .filterBounds(output_geom) \
    .filterMetadata('CLOUD_COVER_LAND', 'less_than', 70)

# l7_coll = ee.ImageCollection('LANDSAT/LE07/C01/T1_RT_TOA') \
#     .filterDate(start_date, end_date) \
#     .filterBounds(output_geom) \
#     .filterMetadata('CLOUD_COVER_LAND', 'less_than', 70)
# l8_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA') \
#     .filterDate(start_date, end_date) \
#     .filterBounds(output_geom) \
#     .filterMetadata('CLOUD_COVER_LAND', 'less_than', 70)
# landast_coll = ee.ImageCollection(l7_coll.merge(l8_coll))

#     .filterMetadata('WRS_PATH', 'equals', 44) \
#     .filterMetadata('WRS_ROW', 'less_than', 35) \
#     .filterMetadata('WRS_ROW', 'greater_than', 32)
#     .filterMetadata('WRS_ROW', 'equals', 33)

pprint.pprint(list(landsat_coll.aggregate_histogram('system:index').getInfo().keys()))

['LC08_043032_20150704',
 'LC08_043032_20150720',
 'LC08_043033_20150720',
 'LC08_043034_20150704',
 'LC08_043034_20150720',
 'LC08_044032_20150711',
 'LC08_044032_20150727',
 'LC08_044033_20150711',
 'LC08_044033_20150727',
 'LC08_044034_20150711',
 'LC08_044034_20150727',
 'LC08_045032_20150702',
 'LC08_045032_20150718',
 'LC08_045033_20150702',
 'LC08_045033_20150718',
 'LC08_045034_20150718']


### Functions

In [7]:
def date_range(start_dt, end_dt, days=1, skip_leap_days=False):
    """Generate dates within a range (inclusive)

    Parameters
    ----------
    start_dt : datetime
        Start date.
    end_dt : datetime
        End date (inclusive).
    days : int, optional
        Step size (the default is 1).
    skip_leap_days : bool, optional
        If True, skip leap days while incrementing (the default is False).

    Yields
    ------
    datetime

    """
    import copy
    curr_dt = copy.copy(start_dt)
    while curr_dt <= end_dt:
        if not skip_leap_days or curr_dt.month != 2 or curr_dt.day != 29:
            yield curr_dt
        curr_dt += datetime.timedelta(days=days)

In [8]:
def get_affine_transform(image):
    return ee.List(ee.Dictionary(ee.Algorithms.Describe(image.projection())).get('transform'))

### Coarse Ta Function

In [9]:
def ta_landsat_func(l_img):
    input_img = ee.Image(landsat.Landsat(l_img).prep())

    # Use the CONUS ALEXI ET but the global landcover and elevation products
    d_obj = disalexi.Image(
        input_img, 
        iterations=10,
        elevation=ee.Image('USGS/SRTMGL1_003').rename(['elevation']),
        landcover=ee.Image(ee.ImageCollection('users/cgmorton/GlobeLand30')
                               .filterBounds(l_img.geometry().bounds(1)).mosaic()) \
            .divide(10).floor().multiply(10).rename(['landcover']),
        lc_type='GLOBELAND30',
        tair_values=list(range(273, 321, 1)),
    )
    
    return d_obj.compute_ta()
    # landsat_crs = landsat_img.select(['B2']).projection().crs()
    # landsat_geo = get_affine_transform(landsat_img.select(['B2']))
    # return d_obj.compute_ta().reproject(crs=landsat_crs, crsTransform=landsat_geo)
    
    
def ta_coarse_func(l_img):
    input_img = ee.Image(landsat.Landsat(l_img).prep())

    # Use the CONUS ALEXI ET but the global landcover and elevation products
    d_obj = disalexi.Image(
        input_img, 
        iterations=10,
        elevation=ee.Image('USGS/SRTMGL1_003').rename(['elevation']),
        landcover=ee.Image(ee.ImageCollection('users/cgmorton/GlobeLand30')
                           .filterBounds(l_img.geometry().bounds(1)).mosaic()) \
            .divide(10).floor().multiply(10).rename(['landcover']),
        lc_type='GLOBELAND30',
        tair_values=list(range(273, 321, 1)),
    )
    
    landsat_crs = landsat_img.select(['B2']).projection().crs()
    landsat_geo = get_affine_transform(landsat_img.select(['B2']))
    
    return d_obj.compute_ta()\
        .reproject(crs=landsat_crs, crsTransform=landsat_geo)\
        .reduceResolution(reducer=ee.Reducer.mean(), maxPixels=65535) \
        .reproject(crs=output_crs, crsTransform=output_geo) \
        .updateMask(1)

### Ta Coarse Export

In [10]:
# for export_dt in date_range(start_dt, end_dt):
export_dt = datetime.datetime(2015, 7, 11)
export_date = export_dt.strftime('%Y-%m-%d')
next_date = (export_dt + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
# print(export_date)

# asset_id = asset_id_fmt.format(collection_id=ta_coarse_id, 
#                                date=export_dt.strftime(asset_dt_fmt))
# task_id = task_id_fmt.format(date=export_dt.strftime(asset_dt_fmt))
# ta_coarse_coll = landsat_coll\
#     .filterDate(export_date, next_date)\
#     .map(ta_coarse_func)
# ta_coarse_img = ee.ImageCollection(ta_coarse_coll)\
#     .mean()\

asset_id = '{coll}/{image}_{cs}'.format(
    coll=ta_coarse_id, image=landsat_id, cs=output_cs_str)
task_id = 'disalexi_tair_coarse_{image}_{cs}'.format(
    image=landsat_id, cs=output_cs_str)
ta_coarse_img = ta_coarse_func(landsat_img)\
    .setMulti({
        'DATE_INGESTED': datetime.datetime.now().strftime('%Y-%m-%d'),
        'DISALEXI_VERSION': openet.disalexi.__version__,
        'DATE': export_date,
    })

print(ta_coarse_img.getInfo())

task = ee.batch.Export.image.toAsset(
    image=ee.Image(ta_coarse_img).toFloat(),
    description=task_id,
    assetId=asset_id,
    crs=output_crs,
    crsTransform=output_geo_str,
    dimensions=output_shape_str,
)
# task.start()
# time.sleep(1)
# print(task.status())
  
#     break

{'type': 'Image', 'bands': [{'id': 't_air', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [0.002, 0.0, -123.0, 0.0, -0.002, 40.0]}], 'properties': {'DATE': '2015-07-11', 'DISALEXI_VERSION': '0.0.2', 'DATE_INGESTED': '2018-11-01'}}


### Ta (Landsat) Export

In [ ]:
export_dt = datetime.datetime(2015, 7, 11)
export_date = export_dt.strftime('%Y-%m-%d')
next_date = (export_dt + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

# l8_coll_id = 'LANDSAT/LC08/C01/T1_RT_TOA'
# l8_coll = ee.ImageCollection(l8_coll_id) \
#     .filterDate(start_date, end_date) \
#     .filterBounds(ee.Geometry.Rectangle([-123, 35, -118.5, 40])) \
#     .filterMetadata('CLOUD_COVER_LAND', 'less_than', 70)
# export_id_list = list(l8_coll.aggregate_histogram('system:index').getInfo().keys())
# # print(export_id_list)

# for export_id in export_id_list:
    
export_id = 'LC08_044033_20150711'
print(export_id)
export_img = ee.Image('{}/{}'.format(l8_coll_id, export_id))

# Switch output cellsize to 60m
export_cs = 60
export_crs = export_img.select(['B2']).projection().getInfo()['crs']
export_geo = export_img.select(['B2']).projection().getInfo()['transform']
export_geo[0] = export_cs
export_geo[4] = -export_cs
export_shape = export_img.select(['B2']).getInfo()['bands'][0]['dimensions']
export_shape[0] = int(export_shape[0] / (export_cs / 30) + 0.5)
export_shape[1] = int(export_shape[1] / (export_cs / 30) + 0.5)
# print(export_crs)
# print(export_geo)
# print(export_shape)

asset_id = '{coll}_{cs}m/{image}_{cs}m'.format(
    coll=ta_landsat_id, image=export_id, cs=export_cs)
task_id = 'disalexi_tair_landsat_{image}_{cs}m'.format(
    image=export_id, cs=export_cs)

ta_landsat_img = ta_landsat_func(export_img)\
    .setMulti({
        'DATE_INGESTED': datetime.datetime.now().strftime('%Y-%m-%d'),
        'DISALEXI_VERSION': openet.disalexi.__version__,
        'DATE': export_date,
    })
task = ee.batch.Export.image.toAsset(
    image=ee.Image(ta_landsat_img).toFloat(),
    description=task_id,
    assetId=asset_id,
    crs=export_crs,
    crsTransform='[' + ','.join(list(map(str, export_geo))) + ']',
    dimensions='{0}x{1}'.format(*export_shape),
)
# task.start()
# time.sleep(1)
# print(task.status())
    
#     break

In [ ]:
export_id = 'LC08_044033_20150711'
export_img = ee.Image('{}/{}'.format(l8_coll_id, export_id))

# Switch output cellsize to 60m
export_cs = 30
export_crs = export_img.select(['B2']).projection().getInfo()['crs']
export_geo = export_img.select(['B2']).projection().getInfo()['transform']
export_geo[0] = export_cs
export_geo[4] = -export_cs
export_shape = export_img.select(['B2']).getInfo()['bands'][0]['dimensions']
export_shape[0] = int(export_shape[0] / (export_cs / 30) + 0.5)
export_shape[1] = int(export_shape[1] / (export_cs / 30) + 0.5)
# print(export_crs)
# print(export_geo)
# print(export_shape)

asset_id = '{coll}_{cs}m/{image}_{cs}m'.format(
    coll=ta_landsat_id, image=export_id, cs=export_cs)
task_id = 'disalexi_tair_landsat_{image}_{cs}m'.format(
    image=export_id, cs=export_cs)

ta_landsat_img = ta_landsat_func(export_img)\
    .setMulti({
        'DATE_INGESTED': datetime.datetime.now().strftime('%Y-%m-%d'),
        'DISALEXI_VERSION': openet.disalexi.__version__,
        'DATE': export_date,
    })
task = ee.batch.Export.image.toAsset(
    image=ee.Image(ta_landsat_img).toFloat(),
    description=task_id,
    assetId=asset_id,
    crs=export_crs,
    crsTransform='[' + ','.join(list(map(str, export_geo))) + ']',
    dimensions='{0}x{1}'.format(*export_shape),
)
task.start()

### Thumbnails

In [ ]:
# thumbnail_crs = 'EPSG:4326'
# thumbnail_cs = 0.005
thumbnail_crs = 'EPSG:32610'
thumbnail_cs = 120
thumbnail_xy = output_geom.bounds(1, thumbnail_crs).coordinates().getInfo()[0]
thumbnail_xmin = int(min(x for x, y in thumbnail_xy) / thumbnail_cs) * thumbnail_cs
thumbnail_ymin = int(min(y for x, y in thumbnail_xy) / thumbnail_cs) * thumbnail_cs
thumbnail_xmax = int(max(x for x, y in thumbnail_xy) / thumbnail_cs) * thumbnail_cs + thumbnail_cs
thumbnail_ymax = int(max(y for x, y in thumbnail_xy) / thumbnail_cs) * thumbnail_cs + thumbnail_cs
thumbnail_geo = [thumbnail_cs, 0.0, thumbnail_xmin, 0.0, thumbnail_cs, thumbnail_ymax]
thumnbail_region = [[], [], [], []]
thumbnail_shape_str = '{0}x{1}'.format(
    int(abs(thumbnail_xmax - thumbnail_xmin) / thumbnail_cs),
    int(abs(thumbnail_ymax - thumbnail_ymin) / thumbnail_cs))
print(thumbnail_crs)
print(thumbnail_geo)
print(thumbnail_shape_str)

In [ ]:
landsat_url = landsat_img.select(['B4', 'B3', 'B2'])\
    .reproject(crs=thumbnail_crs, crsTransform=thumbnail_geo)\
    .getThumbURL({'region': output_region, 'min': 0, 'max': 0.30})
# print(landsat_url)
Image(url=landsat_url)

In [ ]:
landsat_url = landsat_coll.filterDate('2015-07-11', '2015-07-12')\
    .median().select(['B4', 'B3', 'B2'])\
    .reproject(crs=thumbnail_crs, crsTransform=thumbnail_geo)\
    .getThumbURL({'region': output_region, 'min': 0, 'max': 0.30})
# print(landsat_url)
Image(url=landsat_url)

In [ ]:
landsat_url = landsat_coll.filterDate('2015-07-11', '2015-07-12')\
      .mean().select(['B4', 'B3', 'B2'])\
      .reproject(crs=output_crs, crsTransform=output_geo)\
      .getThumbURL({'region': output_region, 'min': 0, 'max': 0.30})
    # print(landsat_url)
Image(url=landsat_url)

### Ta Image

In [ ]:
# ta_landsat_url = ta_landsat_func(landsat_img)\
#     .reproject(crs=thumbnail_crs, crsTransform=thumbnail_geo)\
#     .getThumbURL({
#         'region': output_region, 'min': 273, 'max': 325, 
#         'palette': ','.join(['FF0000', 'FFFF00', '00FFFF', '0000FF'])})
# print(ta_landsat_url)
# Image(url=ta_landsat_url)

In [ ]:
# ta_coarse_url = ta_coarse_func(landsat_img)\
#     .reproject(crs=thumbnail_crs, crsTransform=thumbnail_geo)\
#     .getThumbURL({'region': output_region, 'min': 273, 'max': 325, 
#                   'palette': ','.join(['FF0000', 'FFFF00', '00FFFF', '0000FF'])})
# print(ta_coarse_url)
# Image(url=ta_coarse_url)